In [29]:
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def model(datapath):
    df = pd.read_csv(datapath)
    df_s = df.sort_values('label')
    y = df_s["label"].values
    X = df_s.drop("label",axis=1).values
    #print(df.head())
    #print(df.shape)
    print(y)
    print(X)
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33)

In [30]:
DATAPATH = "data/digits.csv"
model(DATAPATH)

[0 0 0 ... 9 9 9]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
